### Import packages

Import of the packages that will be needed for the project.  This includes packages for data manipulation, sklearn modules and custom functions.

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer

from Project_Functions import offensive_contribution, trailing_stats_mean, tier_maker, get_tiers, get_touchdowns, stats_for_trailing

In [2]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action = 'ignore',
                     category = SettingWithCopyWarning)

### Import Data

Let's import the dataframe that we will be using for modelling

In [3]:
data = pd.read_csv('Data/weekly_data.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0


In [4]:
data.isna().sum()

Unnamed: 0      0
Unnamed: 0.1    0
GameKey         0
PlayerID        0
Season          0
               ..
HomePoints      0
HomeYards       0
AwayTeam        0
AwayPoints      0
AwayYards       0
Length: 65, dtype: int64

Before the train test split, we have to calculate the trailing average fantasy points for each observation, as we cannot incorporate this step into the pipeline without causing data leakage.

In [5]:
def trailing_stats(df):
    
    """
    Function to create a new column with a trailing aggregate mean
    as a new feature for prediction.
    
    Inputs:
        - df: The dataframe on which the function will be applied
        - Column: The column on which to apply the function
        - Window: The number of past values to consider when apply the function

        
    Output:
        - An aggregate value
        
    """
    #Access the column names in stats_for_trailing
    global stats_for_trailing
    
    # Get all unique players in the DataFrame
    players = df['Name'].unique().tolist()
    
    # Define a DataFrame to hold our values
    df_out = pd.DataFrame()
    # Loop through the unique players
    for player in players:
        # Create a temporary dataframe for each player
        temp_df = df[df['Name'] == player]
        # Calculate the n game trailing average for all players.  Set closed parameter to 'left'
        # so that the current value for fantasy points is not included in the calculation.
        # Backfill the two resulting NaN values
        for column in stats_for_trailing:
            temp_df[f'TA{column}'] = temp_df.loc[:,column].rolling(window = 5, 
                                                              closed = 'left').mean().fillna(method = 'bfill')
        # Append the temporary dataframe to the output
        df_out = df_out.append(temp_df)
    # Return a dataframe with the values sorted by the original index
    return df_out.sort_index()
    

In [6]:
stats_for_trailing = ['FantasyPointsPPR']

In [7]:
# Prepare the trailing average fantasy points column

data = trailing_stats(data)
data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards,TAFantasyPointsPPR
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,20.088
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,28.600
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,25.204
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0,18.240
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0,22.420


### Train Test Split

In [8]:
# Separate data from the target
y = data['FantasyPointsPPR']
data.drop(columns = ['FantasyPointsPPR'],
            inplace = True)
X = data

In [9]:
# Execute the train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.25,
                                                   random_state = 13)

### Feature Engineering

The main features that we will be engineering to predict a player's fantasy output will be the 5-game trailing average of various statistics as well as the binning of players into their respective tiers based on recent performance.

In [10]:
# Define the columns for which we want a 5 game trailing average.

stats_for_trailing = ['TotalTouchdowns','RushingYards','PassingInterceptions','PassingTouchdowns','PassingRating','PassingYards',
                      'PassingCompletionPercentage','RushingYards', 'RushingTouchdowns', 'RushingLong',
                      'Receptions','ReceivingYards','ReceivingTargets', 'ReceivingTouchdowns', 'ExtraPointsMade', 'FieldGoalsMade',
                      'Fumbles','FumblesLost']

trailing_stats = []
for col in stats_for_trailing:
    trailing_stats.append('TA' + col)
trailing_stats

['TATotalTouchdowns',
 'TARushingYards',
 'TAPassingInterceptions',
 'TAPassingTouchdowns',
 'TAPassingRating',
 'TAPassingYards',
 'TAPassingCompletionPercentage',
 'TARushingYards',
 'TARushingTouchdowns',
 'TARushingLong',
 'TAReceptions',
 'TAReceivingYards',
 'TAReceivingTargets',
 'TAReceivingTouchdowns',
 'TAExtraPointsMade',
 'TAFieldGoalsMade',
 'TAFumbles',
 'TAFumblesLost']

In [11]:
# Instantiate the function transformers for the feature engineering pipeline

touchdown_transformer = FunctionTransformer(get_touchdowns) # Get total touchdowns per player
trailing_transformer = FunctionTransformer(trailing_stats_mean) # Get the 5 game trailing averages of appropriate statistics
tier_transformer = FunctionTransformer(get_tiers) # Bin players into the appropriate tiers based on recent performance


# Instantiate the pipeline for the necessary transformations

engineering = Pipeline([('touchdown', touchdown_transformer),
                       ('trailing', trailing_transformer),
                       ('tier', tier_transformer)])

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23653 entries, 0 to 31537
Data columns (total 84 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     23653 non-null  int64  
 1   Unnamed: 0.1                   23653 non-null  int64  
 2   GameKey                        23653 non-null  int64  
 3   PlayerID                       23653 non-null  int64  
 4   Season                         23653 non-null  int64  
 5   GameDate                       23653 non-null  object 
 6   Week                           23653 non-null  int64  
 7   Team                           23653 non-null  object 
 8   Opponent                       23653 non-null  object 
 9   HomeOrAway                     23653 non-null  object 
 10  Number                         23653 non-null  int64  
 11  Name                           23653 non-null  object 
 12  Position                       23653 non-null 

<br>

### Preprocessing

As shown above, the bulk of the null values fall into one of two categories.  They are either:
* In the InjuryStatus column
    * Here we can impute a value of healthy, as the only values in the injury column are 
* In the TA (trailing average) columns we created
    * No player with a null value played more than 5 games, therefore we cannot calculate the trailing average for them.  We will impute a default value of 0 for these columns, as they represent players who likely did not have much impact.  If they had an impact, they likely would have played in more games.  I will explore imputing the median value as well through a grid search.

In [16]:
# Define the groups of columns for preprocessing steps.

categorical_columns = ['Week',
                       'Team',
                      'Opponent',
                      'PlayerTier',
                      'InjuryStatus']
numerical_columns = trailing_stats


In [17]:
# Define the preprocessing steps for categorical features
categorical_transform = Pipeline([('impute_cat',SimpleImputer(strategy = 'constant',
                                                              fill_value = 'Healthy')),
                                 ('one_hot_encoder', OneHotEncoder())])

# Define the preprocessing steps for numerical features
numerical_transform = Pipeline([('impute_num', SimpleImputer(strategy = 'constant',
                                                             fill_value = 0)),
                               ('scaler', StandardScaler())])

# Instantiate the column transformer object for the preprocessing steps
preprocessing = ColumnTransformer([('num', numerical_transform, numerical_columns),
                                  ('cat', categorical_transform, categorical_columns)])

In [20]:
# Instantiate a pipeline with a linear regression model as a baseline

pipeline = Pipeline([('engineering', engineering),
                    ('prep', preprocessing),
                    ('model', RandomForestRegressor())])

In [21]:
# Set param grid values, parameters for grid search 

param_grid = {'model__n_estimators': [50, 75, 100],
             'model__max_depth': [4, 5, 6]}

grid_search = GridSearchCV(pipeline, 
                          param_grid = param_grid,
                          cv = 3)

In [22]:
# Fit the grid search to X_train and y_train

grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('engineering',
                                        Pipeline(steps=[('touchdown',
                                                         FunctionTransformer(func=<function get_touchdowns at 0x7fd594919a60>)),
                                                        ('trailing',
                                                         FunctionTransformer(func=<function trailing_stats_mean at 0x7fd594919550>)),
                                                        ('tier',
                                                         FunctionTransformer(func=<function get_tiers at 0x7fd5949199d0>))])),
                                       ('prep',
                                        ColumnTransformer(transform...
                                                                          'TAFieldGoalsMade',
                                                                          'TAFumbles',
                                

In [23]:
grid_search.best_score_

-0.0009927446825482484

In [24]:
preds = grid_search.predict(X_test)

In [25]:
mean_squared_error(y_test, preds)

58.31851714240684

In [26]:
mean_absolute_error(y_test, preds)

5.957462236316819

Let's dig into the predictions a little bit

In [27]:
y_test

22597     9.0
30981     0.0
14444    19.5
20883     0.0
21136     9.8
         ... 
7713      0.0
9304      0.0
22562    16.0
21120     6.8
28390     2.3
Name: FantasyPointsPPR, Length: 7885, dtype: float64

In [28]:
df_preds = pd.DataFrame(y_test)

In [29]:
df_preds['Predicted'] = preds

In [33]:
df_preds[:50].sort_values(by = 'Predicted', ascending = False)

,FantasyPointsPPR,Predicted
11479,0.00,7.702086
14236,0.00,6.089954
24398,2.80,5.844874
19997,24.74,5.818603
13250,0.00,5.812071
15286,0.70,5.785077
14276,0.00,5.783938
3195,16.00,5.766087
30987,0.00,5.708916
1410,0.00,5.698768
